# Applied Product Quantization

### Notebook written by Ryan Awad

Is it feasible to use PQ in EigenDB and if so, how do we go about it?  

Things to look into:
- Training the quantizer with k-means

In [92]:
import numpy as np
from math import ceil, log2

Defining some useful functions

In [93]:
def L2(v: np.ndarray, u: np.ndarray) -> np.float32: # squared distance
    return np.sum((v - u)**2)

'''
c_j is the SET of centroids for the specific sub-vector u_j

this function finds the nearest centroid to a sub-vector and return's it's index (repoduction value)
'''
def quantizer(c_j: np.ndarray[np.ndarray], u_j: np.ndarray, k_: int) -> int:
    distance = float('inf')
    for i in range(k_):
        new_dist = L2(c_j[i], u_j)
        if new_dist < distance:
            reprod_val = i
            distance = new_dist
    return reprod_val

Notice how each value in a vector is a float32

In [94]:
dim = 12
num_vectors = 100
uncompressed_vectors = np.array([np.random.uniform(-100.0, 100.0, dim) for i in range(num_vectors)], dtype=np.float32)
uncompressed_vectors[0][0].dtype

dtype('float32')

In [95]:
m = 4 # dimensions of a compressed vector

assert dim % m == 0 # ensure dim is divisable by m

D_ = int(dim / m) # length of each subvector will be dim / m (D* in notation)
D_

3

In [96]:
# now create the subvectors
u_vectors = []
for v in uncompressed_vectors:
    u = [v[row:row+D_] for row in range(0, dim, D_)]
    u_vectors.append(u)

u_vectors[0]

[array([ 95.237755, -74.24087 ,  84.07467 ], dtype=float32),
 array([ 52.410015 , -56.440662 ,   7.3220177], dtype=float32),
 array([-27.519796, -64.44191 , -35.76187 ], dtype=float32),
 array([ -0.6291111,  96.73161  , -73.86947  ], dtype=float32)]

Creating the codebook (clusters of centroids)

IMPORTANT:
- `k`: total number of centroids generated
- `k_`: number of centroids to choose from per sub-vector

Notice how when you increase `k`, `k_` will increase proportionally. And as `k_` increases, the compressed vector size also increases, WHILE the distances between the regenerated vectors and the actual vectors decrease. In addition, as `k_` increases, so does the time needed to generate the centroids, as well as compressing a vector.

This is because as `k_` increases, each sub-vector gets to choose from a large amount of randomly generated centroids. Probablistically, a sub-vector will find a much better centroid than if `k_` was smaller which would give each sub-vector less options. This causes the average distance between regenerated vectors and actual vectors to decrease.

In addition, as `k_` increases, each sub-vector will have a larger amount of centroids to choose from, making the compression algorithm take longer

On the other hand, each sub-vector is converted into a centroid ID, as part of the compression algorithm. Since each sub-vector gets `k_` centroids to choose from, each ID will need to have allocated memory to store a value between 0 to `k_`-1. Therefore, as `k_` increases, the overall size of a compressed vector will increase. 

In [97]:
k = 2**10
assert k % m == 0
k_ = int(k/m)
print(f"{k=}, {k_=}")
#print(f"Compressed vector size: {ceil(log2(k_)/8) * m} bytes")
#print(f"Centroid map size: {ceil(k*D_*(32/8))} bytes")

k=1024, k_=256


In [98]:
c = []  # codebook

for j in range(m):
    c_j = []
    for i in range(k_):
        c_ji = np.array(np.random.uniform(-100.0, 100.0, D_), dtype=np.float32)
        c_j.append(c_ji)
    c.append(c_j)

In [99]:
compressed_vectors = []
for u in u_vectors:
    curr_comp_vec = []
    for j in range(m):
        reprod_val = quantizer(c[j], u[j], k_)
        curr_comp_vec.append(reprod_val)
    compressed_vectors.append(curr_comp_vec)

print(uncompressed_vectors[0], '->', compressed_vectors[0])

[ 95.237755  -74.24087    84.07467    52.410015  -56.440662    7.3220177
 -27.519796  -64.44191   -35.76187    -0.6291111  96.73161   -73.86947  ] -> [73, 162, 95, 236]


Defining a function to create a distance table for a given uncompressed vector

This gives us the ability to do distance comparisons with a uncompressed vector in the compressed space

In [100]:
def create_distance_table(uncompressed_vector: np.ndarray) -> np.ndarray:
    u = [uncompressed_vector[row:row+D_] for row in range(0, dim, D_)]
    distance_table = np.zeros((k_, m))
    for i in range(k_):
        for j in range(m):
            distance_table[i][j] = L2(u[j], c[j][i])
    return distance_table

random_query_vector = np.array(np.random.uniform(-100.0, 100.0, dim), dtype=np.float32)
distance_table = create_distance_table(random_query_vector)
distance_table

array([[24169.1484375 , 29064.4921875 ,  9789.89160156, 19856.9453125 ],
       [15217.28515625, 23513.99414062, 32702.08203125, 25486.49804688],
       [16749.4921875 , 27566.390625  , 17588.91601562, 39018.8046875 ],
       ...,
       [ 1924.79418945, 16783.17773438,  8501.57128906, 34256.4921875 ],
       [16371.62109375, 20079.5390625 , 33898.39453125, 35699.234375  ],
       [42122.5234375 , 17407.2421875 , 11267.71386719, 37905.22265625]])

Defining two naive $\mathcal{O}(n \log n)$ KNN algorithms. One using PQ compression and the other using uncompressed vectors

In [101]:
def knn_pq(query_vector: np.ndarray, k: int) -> np.ndarray:
    distance_table = create_distance_table(query_vector)
    k_nearest = []
    for v in compressed_vectors:
        dist = 0
        for i in range(len(v)):
            dist += distance_table[v[i]][i]
        k_nearest.append((dist, v))
    k_nearest.sort(key=lambda x: x[0])
    return k_nearest[:k]

def knn(query_vector: np.ndarray, k: int) -> np.ndarray:
    k_nearest = []
    for v in uncompressed_vectors:
        dist = L2(query_vector, v)
        k_nearest.append((dist, v))
    k_nearest.sort(key=lambda x: x[0])
    return k_nearest[:k]
    

In [102]:
knn_pq(random_query_vector, 5)

[(np.float64(17564.37664794922), [238, 133, 100, 41]),
 (np.float64(19173.06591796875), [89, 190, 94, 91]),
 (np.float64(32791.091369628906), [19, 199, 33, 153]),
 (np.float64(34994.06628417969), [239, 249, 218, 193]),
 (np.float64(35544.838134765625), [92, 5, 79, 57])]

In [103]:
knn(random_query_vector, 5)

[(np.float32(15623.364),
  array([-66.87325  , -60.600025 ,   1.1786443,  37.653828 ,  34.695995 ,
          39.55513  ,  27.45343  , -26.210546 , -14.572233 , -21.175188 ,
         -38.014748 ,  78.05014  ], dtype=float32)),
 (np.float32(16095.917),
  array([-97.652664, -94.322845, -25.518024,  74.597916,  62.827755,
          58.08905 ,  14.131105, -33.43935 , -72.77122 , -84.388336,
          57.238464,  30.765211], dtype=float32)),
 (np.float32(28077.338),
  array([-91.153046 ,   5.4039855, -66.23831  ,  44.655132 ,  86.018684 ,
          39.98331  , -91.51644  , -93.11759  , -72.24697  , -31.703691 ,
         -43.683704 , -35.177517 ], dtype=float32)),
 (np.float32(29262.8),
  array([-75.76523 , -31.703144, -34.691883,  40.595818,  33.898514,
         -82.49346 , -55.423103, -42.09285 , -49.451145, -47.776924,
         -21.468914,   6.153912], dtype=float32)),
 (np.float32(29480.121),
  array([-78.11101  ,  80.12574  , -30.671104 ,  63.099766 ,  59.020782 ,
           1.6821917,  